In [3]:
%load_ext nb_black

import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
from datetime import datetime
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import utils

from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator
import jupyter_notebook.stats_t as stats
import matplotlib.pyplot as plt
import warnings
import sklearn.metrics as metrics

# from ipython_utils import cal_acc, show_results
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.stats import percentileofscore
from sklearn.metrics import roc_auc_score, accuracy_score

warnings.filterwarnings("ignore")

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [4]:
disease = "pneumothorax"
path = f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/{disease}"
test_attr = torch.load(os.path.join(path, "dataset_g", "test_attributes.pt"))
test_proba_concept = torch.load(os.path.join(path, "dataset_g", "test_proba_concepts.pt"))
test_logits_concept = torch.load(os.path.join(path, "dataset_g", "test_logits_concepts.pt"))
test_logits_bb = torch.load(os.path.join(path, "dataset_g", "test_logits_bb.pt"))
test_class_labels = torch.load(os.path.join(path, "dataset_g", "test_class_labels.pt"))

train_attr = torch.load(os.path.join(path, "dataset_g", "train_attributes.pt"))
train_proba_concept = torch.load(os.path.join(path, "dataset_g", "train_proba_concepts.pt"))
train_logits_concept = torch.load(os.path.join(path, "dataset_g", "train_logits_concepts.pt"))
train_logits_bb = torch.load(os.path.join(path, "dataset_g", "train_logits_bb.pt"))
train_class_labels = torch.load(os.path.join(path, "dataset_g", "train_class_labels.pt"))

val_attr = torch.load(os.path.join(path, "dataset_g", "val_attributes.pt"))
val_proba_concept = torch.load(os.path.join(path, "dataset_g", "val_proba_concepts.pt"))
val_logits_concept = torch.load(os.path.join(path, "dataset_g", "val_logits_concepts.pt"))
val_logits_bb = torch.load(os.path.join(path, "dataset_g", "val_logits_bb.pt"))
val_class_labels = torch.load(os.path.join(path, "dataset_g", "val_class_labels.pt"))

print(
    test_attr.size(), test_proba_concept.size(), test_logits_concept.size(),
    test_logits_bb.size(), test_class_labels.size()
)
print()
print(
    train_attr.size(), train_proba_concept.size(), train_logits_concept.size(),
    train_logits_bb.size(), train_class_labels.size()
)
print()
print(
    val_attr.size(), val_proba_concept.size(), val_logits_concept.size(),
    val_logits_bb.size(), val_class_labels.size()
)

with open(os.path.join(path, "MIMIC_test_configs.pkl"), "rb") as input_file:
    args = pickle.load(input_file)

concepts = args.landmark_names_spec + args.abnorm_obs_concepts


torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([48144]) torch.Size([24072, 1])

torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([375096]) torch.Size([187548, 1])

torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([46898]) torch.Size([23449, 1])


<IPython.core.display.Javascript object>

In [5]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
sub_auroc = []
sub_recall = []
all_auroc = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    all_auroc.append(auroc_val)
    if auroc_val >= 0.7:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        sub_auroc.append(auroc_val)
        sub_recall.append(recall_val)
        cnt += 1

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

hilar_unspec(3), auroc: 0.7243690504590226, recall: 0.509771986970684
left_pleural(4), auroc: 0.8210022858236588, recall: 0.6498815165876777
right_pleural(5), auroc: 0.8473309288911895, recall: 0.6885428253615128
pleural_unspec(6), auroc: 0.8039039099339746, recall: 0.7406986651519455
heart_size(7), auroc: 0.7254066447315376, recall: 0.530560271646859
left_diaphragm(9), auroc: 0.7898331118785493, recall: 0.5686746987951807
right_diaphragm(10), auroc: 0.7923464089596887, recall: 0.5469061876247505
diaphragm_unspec(11), auroc: 0.7631909798081906, recall: 0.5470085470085471
retrocardiac(12), auroc: 0.7285734222361582, recall: 0.46234153616703955
upper_right_lobe(17), auroc: 0.7296862824634855, recall: 0.453781512605042
left_apical_lung(21), auroc: 0.7873661117913936, recall: 0.532258064516129
left_lung_unspec(22), auroc: 0.7250863950397237, recall: 0.4330042313117066
right_upper_lung(25), auroc: 0.7049597323852506, recall: 0.3624161073825503
right_apical_lung(26), auroc: 0.811988389250276

(0.7833196217943171, 0.5743351947822175)

<IPython.core.display.Javascript object>

In [6]:
mask_select = np.array(all_auroc) >= 0.7

torch_mask = torch.from_numpy(mask_select)
torch_mask.sum(), torch_mask.size()

(tensor(70), torch.Size([107]))

<IPython.core.display.Javascript object>

In [7]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
bad_auroc = []
bad_recall = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    if auroc_val <= 0.7:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        cnt += 1
        bad_auroc.append(auroc_val)
        bad_recall.append(recall_val)

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

trachea(0), auroc: 0.6380706811002521, recall: 0.29411764705882354
left_hilar(1), auroc: 0.6863510588020063, recall: 0.36818181818181817
right_hilar(2), auroc: 0.6521003210636784, recall: 0.3333333333333333
heart_border(8), auroc: 0.5947715240503342, recall: 0.24352331606217617
lower_left_lobe(13), auroc: 0.6968755014783657, recall: 0.5125709651257097
upper_left_lobe(14), auroc: 0.6722628191987523, recall: 0.41534391534391535
lower_right_lobe(15), auroc: 0.6677254055916666, recall: 0.3804945054945055
middle_right_lobe(16), auroc: 0.6718327779609877, recall: 0.34705882352941175
left_lower_lung(18), auroc: 0.6494570339445981, recall: 0.5432393693263259
left_mid_lung(19), auroc: 0.6299225556116619, recall: 0.29441624365482233
left_upper_lung(20), auroc: 0.6823025229983058, recall: 0.3838383838383838
right_lower_lung(23), auroc: 0.6892314863157312, recall: 0.4742319127849356
right_mid_lung(24), auroc: 0.6186900971849866, recall: 0.32
left_costophrenic(30), auroc: 0.6946031785102493, recall

(0.7833196217943171, 0.5743351947822175)

<IPython.core.display.Javascript object>

In [8]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
auroc = []
recall = []
for i in range(len(concepts)):
    try:
        auroc.append(metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i]))
        recall.append(metrics.recall_score(data_GT[:, i], y_pred[:, i]))
    except ValueError:
        pass

print(len(auroc))

mean_auroc = np.mean(np.array(auroc))
mean_recall = np.mean(np.array(recall))
mean_auroc, mean_recall

107


(0.7390533596959601, 0.4995901217485915)

<IPython.core.display.Javascript object>

In [9]:
mean_auroc = np.mean(np.array(auroc))
mean_auroc

0.7390533596959601

<IPython.core.display.Javascript object>

In [10]:
cnt = 0
th = 0.7
concept_select = []

test_select_attr = []
test_select_proba_concept = []
test_select_logits_concept = []


val_select_attr = []
val_select_proba_concept = []
val_select_logits_concept = []


train_select_attr = []
train_select_proba_concept = []
train_select_logits_concept = []


test_select_proba_concept = []
for i in range(len(concepts)):
    if auroc[i] >= th:
        test_select_attr.append(test_attr[:, i])
        test_select_proba_concept.append(test_proba_concept[:, i])
        test_select_logits_concept.append(test_logits_concept[:, i])

        val_select_attr.append(val_attr[:, i])
        val_select_proba_concept.append(val_proba_concept[:, i])
        val_select_logits_concept.append(val_logits_concept[:, i])

        train_select_attr.append(train_attr[:, i])
        train_select_proba_concept.append(train_proba_concept[:, i])
        train_select_logits_concept.append(train_logits_concept[:, i])

        concept_select.append(concepts[i])
        cnt += 1

test_select_attr = torch.stack(test_select_attr, dim=1)
test_select_proba_concept = torch.stack(test_select_proba_concept, dim=1)
test_select_logits_concept = torch.stack(test_select_logits_concept, dim=1)
val_select_attr = torch.stack(val_select_attr, dim=1)
val_select_proba_concept = torch.stack(val_select_proba_concept, dim=1)
val_select_logits_concept = torch.stack(val_select_logits_concept, dim=1)
train_select_attr = torch.stack(train_select_attr, dim=1)
train_select_proba_concept = torch.stack(train_select_proba_concept, dim=1)
train_select_logits_concept = torch.stack(train_select_logits_concept, dim=1)


print("Test sizes: ")
print(test_select_attr.size())
print(test_select_proba_concept.size())
print(test_select_logits_concept.size())
print("Val sizes: ")
print(val_select_attr.size())
print(val_select_proba_concept.size())
print(val_select_logits_concept.size())
print("Train sizes: ")
print(train_select_attr.size())
print(train_select_proba_concept.size())
print(train_select_logits_concept.size())
print(cnt)

Test sizes: 
torch.Size([24072, 70])
torch.Size([24072, 70])
torch.Size([24072, 70])
Val sizes: 
torch.Size([23449, 70])
torch.Size([23449, 70])
torch.Size([23449, 70])
Train sizes: 
torch.Size([187548, 70])
torch.Size([187548, 70])
torch.Size([187548, 70])
70


<IPython.core.display.Javascript object>

In [11]:
metric = "auroc"
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"torch_concepts_mask_{metric}.pt"),
    tensor_to_save=torch_mask,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_{metric}_attributes.pt"),
    tensor_to_save=test_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_proba_{metric}_concepts.pt"),
    tensor_to_save=test_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_logits_{metric}_concepts.pt"),
    tensor_to_save=test_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_{metric}_attributes.pt"),
    tensor_to_save=val_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_proba_{metric}_concepts.pt"),
    tensor_to_save=val_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_logits_{metric}_concepts.pt"),
    tensor_to_save=val_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_{metric}_attributes.pt"),
    tensor_to_save=train_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_proba_{metric}_concepts.pt"),
    tensor_to_save=train_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_logits_{metric}_concepts.pt"),
    tensor_to_save=train_select_logits_concept,
)

pickle.dump(
    concept_select,
    open(os.path.join(path, "dataset_g", f"selected_concepts_{metric}.pkl"), "wb"),
)
print(os.path.join(path, "dataset_g"))

/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/pneumothorax/dataset_g


<IPython.core.display.Javascript object>

In [12]:
concept_select

['hilar_unspec',
 'left_pleural',
 'right_pleural',
 'pleural_unspec',
 'heart_size',
 'left_diaphragm',
 'right_diaphragm',
 'diaphragm_unspec',
 'retrocardiac',
 'upper_right_lobe',
 'left_apical_lung',
 'left_lung_unspec',
 'right_upper_lung',
 'right_apical_lung',
 'right_lung_unspec',
 'lung_apices',
 'lung_bases',
 'mediastinal',
 'spine',
 'right_atrium',
 'right_ventricle',
 'aorta',
 'svc',
 'interstitium',
 'parenchymal',
 'cavoatrial_junction',
 'stomach',
 'effusion',
 'edema',
 'atelectasis',
 'tube',
 'consolidation',
 'process',
 'abnormality',
 'enlarge',
 'tip',
 'low',
 'line',
 'congestion',
 'catheter',
 'cardiomegaly',
 'air',
 'tortuous',
 'lead',
 'disease',
 'calcification',
 'device',
 'engorgement',
 'picc',
 'elevation',
 'wire',
 'fluid',
 'degenerative',
 'pacemaker',
 'marking',
 'scar',
 'widen',
 'collapse',
 'emphysema',
 'mass',
 'crowd',
 'infiltrate',
 'obscure',
 'deformity',
 'hernia',
 'drainage',
 'distention',
 'shift',
 'pressure',
 'chf']

<IPython.core.display.Javascript object>

In [13]:
y_gt_tr = torch.load(os.path.join(path, "dataset_g", f"train_class_labels.pt"))
(y_gt_tr == 1).sum(dim=0), (y_gt_tr == 0).sum(dim=0)

(tensor([7107]), tensor([180441]))

<IPython.core.display.Javascript object>

In [14]:
((y_gt_tr == 1).sum(dim=0) / y_gt_tr.size(0)).item()

0.037894297391176224

<IPython.core.display.Javascript object>